In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
import time

In [ ]:
import zipfile

zip_path = "/content/preprocessed.zip"
extract_path = "datasets/preprocessed"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped into datasets/preprocessed/")

Dataset unzipped into datasets/preprocessed/


##  EfficientNetV2 on Pre-processed  Fundus Images

- We use built-in `EfficientNet_V2_S_Weights` transforms for inference-ready preprocessing.

- We initialize an **EfficientNet-V2-S model pretrained on ImageNet** and modify the final classifier layer for **2 classes**:




In [ ]:
# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


# Dataset Paths
train_dir = "/content/datasets/preprocessed/preprocessed/train"
val_dir   = "/content/datasets/preprocessed/preprocessed/val"
test_dir  = "/content/datasets/preprocessed/preprocessed/test"

Using device: cuda


In [ ]:
# Transforms
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
preprocess = weights.transforms()

# Train transform with augmentation
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    preprocess
])

# Validation & Test transform
val_test_transform = preprocess

In [ ]:
# Datasets & Dataloaders
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset   = datasets.ImageFolder(val_dir, transform=val_test_transform)
test_dataset  = datasets.ImageFolder(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

print("Dataset sizes:")
print("Train:", len(train_dataset))
print("Val:", len(val_dataset))
print("Test:", len(test_dataset))

Dataset sizes:
Train: 10122
Val: 708
Test: 712


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Load EfficientNet-V2 Model
model = efficientnet_v2_s(weights=weights)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 2)  # 2 classes

model = model.to(DEVICE)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth


100%|██████████| 82.7M/82.7M [00:00<00:00, 198MB/s]


In [ ]:
# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Training Loop
NUM_EPOCHS = 10
best_acc = 0

for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    model.train()
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    #  Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] | Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f} | Time: {time.time()-start_time:.1f}s")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_efficientnetv2.pth")
        print("Saved new best model:", val_acc)

Epoch [1/10] | Loss: 0.1540 | Val Acc: 0.9718 | Time: 302.9s
Saved new best model: 0.9717514124293786
Epoch [2/10] | Loss: 0.0542 | Val Acc: 0.9732 | Time: 312.4s
Saved new best model: 0.9731638418079096
Epoch [3/10] | Loss: 0.0381 | Val Acc: 0.9746 | Time: 311.8s
Saved new best model: 0.9745762711864406
Epoch [4/10] | Loss: 0.0282 | Val Acc: 0.9802 | Time: 311.8s
Saved new best model: 0.980225988700565
Epoch [5/10] | Loss: 0.0233 | Val Acc: 0.9802 | Time: 311.2s
Epoch [6/10] | Loss: 0.0123 | Val Acc: 0.9774 | Time: 312.2s
Epoch [7/10] | Loss: 0.0201 | Val Acc: 0.9831 | Time: 311.8s
Saved new best model: 0.9830508474576272
Epoch [8/10] | Loss: 0.0213 | Val Acc: 0.9534 | Time: 311.1s
Epoch [9/10] | Loss: 0.0147 | Val Acc: 0.9732 | Time: 311.4s
Epoch [10/10] | Loss: 0.0176 | Val Acc: 0.9590 | Time: 312.4s




###  Final Evaluation
We evaluate the enhanced model on the **test dataset**, giving the final:


In [ ]:
# Test Accuracy
model.load_state_dict(torch.load("best_efficientnetv2.pth"))
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(" TEST ACCURACY:", test_acc)

 TEST ACCURACY: 0.9775280898876404
